In [1]:
from glio.loaders import nifti
from glio.jupyter_tools import show_slices, show_slices_arr
from glio.torch_tools import to_binary, center_of_mass, binary_erode3d
import torch
import polars as pl

In [2]:
clinical_data = pl.read_csv(r"E:\dataset\RHUH-GBM\clinical_data_TCIA_RHUH-GBM.csv")
clinical_data

Patient ID,Days from earliest imaging to surgery,Age,Sex,Preoperative KPS,"Previous treatment = (no, surgery, surgery + QT/RT)",Histopathological subtype,WHO grade,"IDH status = (mutant [mut], wild type [wt], NOS)","Operative adjuncts = (5' aminolevulinic acid [5ALA], Sodium Fluorescein - Yellow 560 [FL], Neuronavigation [NAV], Intraoperative monitoring [IONM], Direct electrical stimulation [DES], intraoperative ultrasound [ioUS])",Preoperative contrast enhancing tumor volume (cm3),Postoperative contrast enhancing residual tumor (cm3),Preoperative T2/FLAIR abnormality (cm3),Postoperative T2/FLAIR abmnormality (cm3),Extent of resection [EOR] %,"EOR = (Gross total resection [GTR : 100%], Near total resection [NTR : > 95%], Subtotal resection [STR : 91 - 94%], Partial resection [PR : < 90 %]","Adjuvant therapy (Radiotherapy [RT], Temozolomide [TMZ])",Radiotherapy treatment details (technique/dose/number of fractions),Postoperative Neurological Deficit,Postoperative KPS,Progression free survival [PFS] (days),Overall survival [OS] (days),Right Censored
str,i64,i64,str,i64,str,str,i64,str,str,f64,f64,f64,f64,f64,str,str,str,str,i64,i64,i64,str
"""RHUH-0001""",1,65,"""male""",80,"""no""","""glioblastoma""",4,"""mut""","""5ALA+NAV+ioUS""",103.21,0.0,36.29,25.17,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,649,736,"""no"""
"""RHUH-0002""",9,65,"""female""",90,"""no""","""glioblastoma""",4,"""wt""","""5ALA+NAV+ioUS""",1.6,0.0,27.06,6.64,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,314,506,"""yes"""
"""RHUH-0003""",6,75,"""male""",80,"""no""","""glioblastoma""",4,"""wt""","""5ALA+NAV+ioUS""",15.26,0.0,26.17,1.05,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",80,135,235,"""no"""
"""RHUH-0004""",5,64,"""female""",90,"""no""","""glioblastoma""",4,"""wt""","""5ALA+NAV+ioUS""",13.15,0.0,105.27,45.38,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,705,947,"""no"""
"""RHUH-0005""",9,52,"""male""",70,"""no""","""glioblastoma""",4,"""wt""","""5ALA+NAV+ioUS""",12.28,0.22,55.3,29.3,98.2,"""NTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""Transient""",70,468,723,"""no"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""RHUH-0036""",19,60,"""female""",70,"""no""","""glioblastoma""",4,"""wt""","""5ALA+NAV+ioUS""",31.57,0.0,141.74,23.65,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,587,841,"""no"""
"""RHUH-0037""",33,45,"""male""",90,"""no""","""glioblastoma""",4,"""mut""","""5ALA+NAV+ioUS""",7.85,0.0,59.73,10.39,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",80,351,656,"""no"""
"""RHUH-0038""",1,69,"""male""",90,"""no""","""glioblastoma""",4,"""wt""","""5ALA+NAV+ioUS""",52.89,0.0,125.46,81.82,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,172,331,"""no"""


In [3]:
PATH = r"E:\dataset\RHUH-GBM\RHUH-GBM_nii_v1"
PATIENT = "RHUH-0001"
OBSERVATION = 2
MOD = "segmentations"
image = nifti.read(fr"{PATH}/{PATIENT}/{OBSERVATION}/{PATIENT}_{OBSERVATION}_{MOD}.nii.gz")
show_slices(image)

interactive(children=(Checkbox(value=False, description='color'), IntSlider(value=119, description='s0', max=2…

<function glio.jupyter_tools.show_slices.<locals>.f(color, **kwargs)>

In [4]:
MOD1 = "segmentations"
MOD2 = "t1ce"
image1 = nifti.read(fr"{PATH}/{PATIENT}/{OBSERVATION}/{PATIENT}_{OBSERVATION}_{MOD1}.nii.gz")
image2 = nifti.read(fr"{PATH}/{PATIENT}/{OBSERVATION}/{PATIENT}_{OBSERVATION}_{MOD2}.nii.gz")
full = torch.stack((image2, image2, image2))
full[0] += image1*2
show_slices_arr(full)

interactive(children=(Checkbox(value=False, description='color'), Text(value='0 1 2 3', description='permute')…

<function glio.jupyter_tools.show_slices_arr.<locals>.f(color, permute: str, **kwargs)>

In [5]:
image = nifti.read(fr"{PATH}/{PATIENT}/{OBSERVATION}/{PATIENT}_{OBSERVATION}_{MOD}.nii.gz")
print(image.min(), image.max())
image = image.where(image<=0, 1)
print(image.min(), image.max())
show_slices(image)

tensor(0, dtype=torch.int32) tensor(3, dtype=torch.int32)
tensor(0, dtype=torch.int32) tensor(1, dtype=torch.int32)


interactive(children=(Checkbox(value=False, description='color'), IntSlider(value=119, description='s0', max=2…

<function glio.jupyter_tools.show_slices.<locals>.f(color, **kwargs)>

In [6]:
image = nifti.read(fr"{PATH}/{PATIENT}/{OBSERVATION}/{PATIENT}_{OBSERVATION}_{MOD}.nii.gz")
com = center_of_mass(to_binary(image)).int()
print(com)
image[com[0], com[1], com[2]] = 10


show_slices_arr(image)

tensor([154, 140,  76], dtype=torch.int32)


interactive(children=(Checkbox(value=False, description='color'), Text(value='0 1 2', description='permute'), …

<function glio.jupyter_tools.show_slices_arr.<locals>.f(color, permute: str, **kwargs)>

In [7]:
clinical_data.row(by_predicate=pl.col("Patient ID") == "RHUH-0001")

('RHUH-0001',
 1,
 65,
 'male',
 80,
 'no',
 'glioblastoma',
 4,
 'mut',
 '5ALA+NAV+ioUS',
 103.21,
 0.0,
 36.29,
 25.17,
 100.0,
 'GTR',
 'RT + TMZ',
 'VMAT-IMRT-IGRT/60 Gy /30 fx',
 'No',
 90,
 649,
 736,
 'no')

In [8]:
from glio.python_tools import get_all_files
import os
patients = []
observation = []
center = []
adc = []
flair = []
segmentations = []
t1 = []
t1ce = []
t2 = []
for i, patient in enumerate(os.listdir(r"E:\dataset\RHUH-GBM\RHUH-GBM_nii_v1")):
    for obs in os.listdir(rf"E:\dataset\RHUH-GBM\RHUH-GBM_nii_v1\{patient}"):
        print(i, end="|")
        patients.append(patient)
        observation.append(obs)
        adc.append(rf"E:\dataset\RHUH-GBM\RHUH-GBM_nii_v1\{patient}\{obs}\{patient}_{obs}_adc.nii.gz")
        flair.append(rf"E:\dataset\RHUH-GBM\RHUH-GBM_nii_v1\{patient}\{obs}\{patient}_{obs}_flair.nii.gz")
        segmentations.append(rf"E:\dataset\RHUH-GBM\RHUH-GBM_nii_v1\{patient}\{obs}\{patient}_{obs}_segmentations.nii.gz")
        t1.append(rf"E:\dataset\RHUH-GBM\RHUH-GBM_nii_v1\{patient}\{obs}\{patient}_{obs}_t1.nii.gz")
        t1ce.append(rf"E:\dataset\RHUH-GBM\RHUH-GBM_nii_v1\{patient}\{obs}\{patient}_{obs}_t1ce.nii.gz")
        t2.append(rf"E:\dataset\RHUH-GBM\RHUH-GBM_nii_v1\{patient}\{obs}\{patient}_{obs}_t2.nii.gz")

        segm = nifti.read(segmentations[-1])
        center.append(center_of_mass(binary_erode3d(to_binary(segm), 2)).tolist())

df = pl.from_dict({"Patient ID": patients, "observation": observation, "center": center, "adc": adc, "flair": flair, "segmentations": segmentations, "t1": t1, "t1ce": t1ce, "t2": t2})
df

0|0|0|1|1|1|2|2|2|3|3|3|4|4|4|5|5|5|6|6|6|7|7|7|8|8|8|9|9|9|10|10|10|11|11|11|12|12|12|13|13|13|14|14|14|15|15|15|16|16|16|17|17|17|18|18|18|19|19|19|20|20|20|21|21|21|22|22|22|23|23|23|24|24|24|25|25|25|26|26|26|27|27|27|28|28|28|29|29|29|30|30|30|31|31|31|32|32|32|33|33|33|34|34|34|35|35|35|36|36|36|37|37|37|38|38|38|39|39|39|

Patient ID,observation,center,adc,flair,segmentations,t1,t1ce,t2
str,str,list[f64],str,str,str,str,str,str
"""RHUH-0001""","""0""","[149.644363, 159.284821, 72.567497]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
"""RHUH-0001""","""1""","[145.472656, 169.080826, 71.031189]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
"""RHUH-0001""","""2""","[154.362396, 140.513153, 76.515083]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
"""RHUH-0002""","""0""","[98.477936, 189.186264, 77.249344]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
"""RHUH-0002""","""1""","[95.099998, 160.711945, 75.162315]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
…,…,…,…,…,…,…,…,…
"""RHUH-0039""","""1""","[96.674034, 103.102638, 90.295456]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
"""RHUH-0039""","""2""","[91.71888, 117.445633, 78.311302]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
"""RHUH-0040""","""0""","[137.862671, 148.065155, 105.572212]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"


In [9]:
merged = clinical_data.join(df, how='outer', on="Patient ID")
merged

Patient ID,Days from earliest imaging to surgery,Age,Sex,Preoperative KPS,"Previous treatment = (no, surgery, surgery + QT/RT)",Histopathological subtype,WHO grade,"IDH status = (mutant [mut], wild type [wt], NOS)","Operative adjuncts = (5' aminolevulinic acid [5ALA], Sodium Fluorescein - Yellow 560 [FL], Neuronavigation [NAV], Intraoperative monitoring [IONM], Direct electrical stimulation [DES], intraoperative ultrasound [ioUS])",Preoperative contrast enhancing tumor volume (cm3),Postoperative contrast enhancing residual tumor (cm3),Preoperative T2/FLAIR abnormality (cm3),Postoperative T2/FLAIR abmnormality (cm3),Extent of resection [EOR] %,"EOR = (Gross total resection [GTR : 100%], Near total resection [NTR : > 95%], Subtotal resection [STR : 91 - 94%], Partial resection [PR : < 90 %]","Adjuvant therapy (Radiotherapy [RT], Temozolomide [TMZ])",Radiotherapy treatment details (technique/dose/number of fractions),Postoperative Neurological Deficit,Postoperative KPS,Progression free survival [PFS] (days),Overall survival [OS] (days),Right Censored,Patient ID_right,observation,center,adc,flair,segmentations,t1,t1ce,t2
str,i64,i64,str,i64,str,str,i64,str,str,f64,f64,f64,f64,f64,str,str,str,str,i64,i64,i64,str,str,str,list[f64],str,str,str,str,str,str
"""RHUH-0001""",1,65,"""male""",80,"""no""","""glioblastoma""",4,"""mut""","""5ALA+NAV+ioUS""",103.21,0.0,36.29,25.17,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,649,736,"""no""","""RHUH-0001""","""0""","[149.644363, 159.284821, 72.567497]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
"""RHUH-0001""",1,65,"""male""",80,"""no""","""glioblastoma""",4,"""mut""","""5ALA+NAV+ioUS""",103.21,0.0,36.29,25.17,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,649,736,"""no""","""RHUH-0001""","""1""","[145.472656, 169.080826, 71.031189]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
"""RHUH-0001""",1,65,"""male""",80,"""no""","""glioblastoma""",4,"""mut""","""5ALA+NAV+ioUS""",103.21,0.0,36.29,25.17,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,649,736,"""no""","""RHUH-0001""","""2""","[154.362396, 140.513153, 76.515083]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
"""RHUH-0002""",9,65,"""female""",90,"""no""","""glioblastoma""",4,"""wt""","""5ALA+NAV+ioUS""",1.6,0.0,27.06,6.64,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,314,506,"""yes""","""RHUH-0002""","""0""","[98.477936, 189.186264, 77.249344]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
"""RHUH-0002""",9,65,"""female""",90,"""no""","""glioblastoma""",4,"""wt""","""5ALA+NAV+ioUS""",1.6,0.0,27.06,6.64,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,314,506,"""yes""","""RHUH-0002""","""1""","[95.099998, 160.711945, 75.162315]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""RHUH-0039""",1,67,"""female""",80,"""no""","""glioblastoma""",4,"""wt""","""5ALA+FL+NAV+io…",44.77,0.0,77.06,59.81,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",80,179,211,"""yes""","""RHUH-0039""","""1""","[96.674034, 103.102638, 90.295456]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
"""RHUH-0039""",1,67,"""female""",80,"""no""","""glioblastoma""",4,"""wt""","""5ALA+FL+NAV+io…",44.77,0.0,77.06,59.81,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",80,179,211,"""yes""","""RHUH-0039""","""2""","[91.71888, 117.445633, 78.311302]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…"
"""RHUH-0040""

In [41]:
def func(x):
    x = list(x)
    x[25] = str(x[25])[1:-1]
    return tuple(x)
merged.with_columns(
                center=pl.col("center").map_elements(lambda x: str(x.to_list())[1:-1]),
                src=pl.lit("center"),).write_csv("merged.csv")

In [45]:
class RHUH_GBM:
    def __init__(self):
        self.clinical_data = pl.read_csv(r"F:\Stuff\Programming\AI\glio_diff\glio\datasets\RHUH-GBM.csv").with_columns(
                center=pl.col("center").map_elements(lambda x: [float(i) for i in str(x[1:-1]).split(", ")]),
                src=pl.lit("center"),)
        self.path = r"E:\dataset\RHUH-GBM\RHUH-GBM_nii_v1"

    def get_data(self, cols):
        return self.clinical_data.select(cols).drop_nulls().to_dicts()

    def col_names(self):
        return self.clinical_data.columns


In [46]:
rhuh = RHUH_GBM()

In [47]:
rhuh.clinical_data

Patient ID,Days from earliest imaging to surgery,Age,Sex,Preoperative KPS,"Previous treatment = (no, surgery, surgery + QT/RT)",Histopathological subtype,WHO grade,"IDH status = (mutant [mut], wild type [wt], NOS)","Operative adjuncts = (5' aminolevulinic acid [5ALA], Sodium Fluorescein - Yellow 560 [FL], Neuronavigation [NAV], Intraoperative monitoring [IONM], Direct electrical stimulation [DES], intraoperative ultrasound [ioUS])",Preoperative contrast enhancing tumor volume (cm3),Postoperative contrast enhancing residual tumor (cm3),Preoperative T2/FLAIR abnormality (cm3),Postoperative T2/FLAIR abmnormality (cm3),Extent of resection [EOR] %,"EOR = (Gross total resection [GTR : 100%], Near total resection [NTR : > 95%], Subtotal resection [STR : 91 - 94%], Partial resection [PR : < 90 %]","Adjuvant therapy (Radiotherapy [RT], Temozolomide [TMZ])",Radiotherapy treatment details (technique/dose/number of fractions),Postoperative Neurological Deficit,Postoperative KPS,Progression free survival [PFS] (days),Overall survival [OS] (days),Right Censored,Patient ID_right,observation,center,adc,flair,segmentations,t1,t1ce,t2,src
str,i64,i64,str,i64,str,str,i64,str,str,f64,f64,f64,f64,f64,str,str,str,str,i64,i64,i64,str,str,i64,list[f64],str,str,str,str,str,str,str
"""RHUH-0001""",1,65,"""male""",80,"""no""","""glioblastoma""",4,"""mut""","""5ALA+NAV+ioUS""",103.21,0.0,36.29,25.17,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,649,736,"""no""","""RHUH-0001""",0,"[49.644363, 159.284821, 72.567497]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""center"""
"""RHUH-0001""",1,65,"""male""",80,"""no""","""glioblastoma""",4,"""mut""","""5ALA+NAV+ioUS""",103.21,0.0,36.29,25.17,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,649,736,"""no""","""RHUH-0001""",1,"[45.472656, 169.080826, 71.031189]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""center"""
"""RHUH-0001""",1,65,"""male""",80,"""no""","""glioblastoma""",4,"""mut""","""5ALA+NAV+ioUS""",103.21,0.0,36.29,25.17,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,649,736,"""no""","""RHUH-0001""",2,"[54.362396, 140.513153, 76.515083]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""center"""
"""RHUH-0002""",9,65,"""female""",90,"""no""","""glioblastoma""",4,"""wt""","""5ALA+NAV+ioUS""",1.6,0.0,27.06,6.64,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,314,506,"""yes""","""RHUH-0002""",0,"[8.477936, 189.186264, 77.249344]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""center"""
"""RHUH-0002""",9,65,"""female""",90,"""no""","""glioblastoma""",4,"""wt""","""5ALA+NAV+ioUS""",1.6,0.0,27.06,6.64,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",90,314,506,"""yes""","""RHUH-0002""",1,"[5.099998, 160.711945, 75.162315]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""center"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""RHUH-0039""",1,67,"""female""",80,"""no""","""glioblastoma""",4,"""wt""","""5ALA+FL+NAV+io…",44.77,0.0,77.06,59.81,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",80,179,211,"""yes""","""RHUH-0039""",1,"[6.674034, 103.102638, 90.295456]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""center"""
"""RHUH-0039""",1,67,"""female""",80,"""no""","""glioblastoma""",4,"""wt""","""5ALA+FL+NAV+io…",44.77,0.0,77.06,59.81,100.0,"""GTR""","""RT + TMZ""","""VMAT-IMRT-IGRT…","""No""",80,179,211,"""yes""","""RHUH-0039""",2,"[1.71888, 117.445633, 78.311302]","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\RHU…","""E:\dataset\R